In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from glob import glob

In [3]:
print(tf. __version__)

2.3.1


In [4]:
print(tf.keras. __version__)

2.4.0


In [3]:
img_rows , img_cols =299,299
train_path = '/home/nimisha/Documents/Rice_2.0/rice_website_v1/train'

In [19]:
InResV2_model= InceptionResNetV2(weights='imagenet', include_top= False, input_shape=(img_rows,img_cols,3))

In [20]:
def addTopModelInResv2(bottom_model,num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1536,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(192,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [5]:
num_classes=11

In [23]:
FC_Head = addTopModelInResv2(InResV2_model, num_classes)

In [24]:
model=Model(inputs = InResV2_model.input,outputs= FC_Head)
print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

Total params: 57,142,507
Trainable params: 57,081,963
Non-trainable params: 60,544
__________________________________________________________________________________________________
None


In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir='/home/nimisha/Documents/Rice_2.0/rice_website_v1/train'
validation_data_dir='/home/nimisha/Documents/Rice_2.0/rice_website_v1/validation'

In [7]:
image_generator =ImageDataGenerator(rescale=1./255)

In [8]:
batch_size =32
train_generator = image_generator.flow_from_directory(train_data_dir,target_size=(img_rows,img_cols),batch_size=batch_size,
                                                   class_mode='categorical')

Found 61620 images belonging to 11 classes.


In [9]:
validation_generator = image_generator.flow_from_directory(validation_data_dir,target_size=(img_rows,img_cols),
                                                              batch_size=batch_size,
                                                   class_mode='categorical')


Found 15420 images belonging to 11 classes.


In [10]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [30]:
#checkpoint = ModelCheckpoint("/home/nimisha/Documents/sample_powerAI/Rice_dataset/models/newRice_InResv2_11_26_40.h5",
                            # monitor="val_loss",mode="min",save_best_only = True,verbose=1)

In [11]:
model.compile(loss= 'categorical_crossentropy',optimizer = Adam(learning_rate = 0.00001), metrics = ['accuracy'])

#model.compile(optimizer = Adam(learning_rate =[1e-2,1e-3]),
                 #loss = 'categorical_crossentropy',
                 #metrics = ['accuracy'])

NameError: name 'model' is not defined

In [20]:
earlystop=EarlyStopping(monitor ='val_loss',min_delta = 0,patience = 8,verbose = 1,restore_best_weights = True)

In [21]:
#callbacks = [earlystop , checkpoint]

In [7]:
nb_train_samples = 61620
nb_validation_samples = 15420
epochs = 25
batch_size = 32

In [32]:
fit = model.fit_generator(train_generator,steps_per_epoch = nb_train_samples // batch_size, epochs = epochs,
                              #callbacks = callbacks, 
                          validation_data = validation_generator,
                              validation_steps = nb_validation_samples // batch_size)

Epoch 1/25
1925/1925 [==============================] - 4588s 2s/step - loss: 0.4515 - accuracy: 0.8486 - val_loss: 0.1391 - val_accuracy: 0.9500
Epoch 2/25
1925/1925 [==============================] - 3200s 2s/step - loss: 0.0700 - accuracy: 0.9755 - val_loss: 0.1807 - val_accuracy: 0.9435
Epoch 3/25
1925/1925 [==============================] - 3206s 2s/step - loss: 0.0349 - accuracy: 0.9886 - val_loss: 0.1389 - val_accuracy: 0.9602
Epoch 4/25
1925/1925 [==============================] - 3206s 2s/step - loss: 0.0211 - accuracy: 0.9930 - val_loss: 0.1416 - val_accuracy: 0.9626
Epoch 5/25
1925/1925 [==============================] - 3203s 2s/step - loss: 0.0145 - accuracy: 0.9953 - val_loss: 0.1361 - val_accuracy: 0.9634
Epoch 6/25
1925/1925 [==============================] - 3205s 2s/step - loss: 0.0104 - accuracy: 0.9967 - val_loss: 0.1174 - val_accuracy: 0.9699
Epoch 7/25
1925/1925 [==============================] - 3211s 2s/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.1163 -

In [13]:
#loss
plt.plot(classifier.history['loss'], label = 'train_loss')
plt.plot(classifier.history['val_loss'], label ='val_loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

NameError: name 'fit' is not defined

In [ ]:
#accuracy
plt.plot(fit.history['accuracy'], label='train_acc')
plt.plot(fit.history['val_accuracy'], label = 'val_acc')
plt.legend()
plt.show()
plt.savefig('AccuracyVal_acc')

In [8]:
import tensorflow as tf
from keras.models import load_model

In [9]:
model.save('/home/nimisha/Documents/Rice_2.0/rice_website_v1/TT_folder sep/Rice_InResv2_Aug30.h5')

NameError: name 'model' is not defined

In [10]:
classifier = load_model('/home/nimisha/Documents/Rice_2.0/rice_website_v1/TT_folder sep/Rice_InResv2_Aug30.h5')

In [11]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np

In [12]:
evaluation_data_dir= '/home/nimisha/Documents/Rice_2.0/rice_website_v1/test_val'
evaluation_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
evaluation_generator = evaluation_datagen.flow_from_directory(evaluation_data_dir,target_size=(img_rows,img_cols),
                                                              batch_size=batch_size,
                                                              classes=['1121','1509','1637','1718','1728','BAS_370','CSR_30','DHBT_3','PB1','PB_6','Unknown'],
                                                   class_mode='categorical', shuffle = False)
true_classes = evaluation_generator.classes
class_labels = list(evaluation_generator.class_indices)   

class_labels

Found 642 images belonging to 11 classes.


['1121',
 '1509',
 '1637',
 '1718',
 '1728',
 'BAS_370',
 'CSR_30',
 'DHBT_3',
 'PB1',
 'PB_6',
 'Unknown']

In [14]:
target_names = ['1121','1509','1637','1718','1728','BAS_370','CSR_30','DHBT_3','PB1','PB_6','Unknown']
Y_pred = classifier.predict_generator(evaluation_generator, 642 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = metrics.confusion_matrix(evaluation_generator.classes, y_pred)
print(cm)
print('Classification Report')
print(metrics.classification_report(evaluation_generator.classes, y_pred))

Instructions for updating:
Please use Model.predict, which supports generators.
Confusion Matrix
[[ 50   0   0   0   0   0   0   0   0   0   0]
 [  0  50   0   0   0   0   0   0   0   0   0]
 [  0   0  46   0   0   0   0   0   4   0   0]
 [  0   0   0  50   0   0   0   0   0   0   0]
 [  3   0   0   0  47   0   0   0   0   0   0]
 [  0   0   0   0   0  50   0   0   0   0   0]
 [  0   0   0   0   0   0  50   0   0   0   0]
 [  0   0   0   0   0   0   0  50   0   0   0]
 [  0   0   0   0   0   0   0   0  50   0   0]
 [  0   0   0   0   0   0   0   0   0  50   0]
 [  0   0   0   0   0   0   0   0   1   0 141]]
Classification Report
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        50
           1       1.00      1.00      1.00        50
           2       1.00      0.92      0.96        50
           3       1.00      1.00      1.00        50
           4       1.00      0.94      0.97        50
           5       1.00      1.00     

In [28]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
img1 = image.load_img('/home/nimisha/Documents/Rice_2.0/test_val/BAS_370/PD_BAS_370_39_6.jpg', target_size=(299, 299))
img = image.img_to_array(img1)
img = img/255
# create a batch of size 1 [N,H,W,C]
img = np.expand_dims(img, axis=0)
prediction = classifier.predict(img, batch_size=None,steps=1) #gives all class prob.
print(prediction)

#1637/PD_1637_8_2.jpg
#1637/PD_1637_19_2.jpg

[[2.8208431e-17 5.6716486e-14 1.0349263e-19 1.6331029e-16 3.6453022e-20
  1.0000000e+00 4.1514500e-12 4.4491856e-16 7.6036894e-14 3.8141665e-15
  1.1632870e-10]]
